# MDN-RNN

MDN-RNN represents the memory (M) of worldmodel architecture. It takes compressed state vector $ Z $ as well as the action and learns the probability distribution of next state using mixture of Gaussians

In [12]:
import numpy as np

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# Build Model

In [4]:
class MDNLayer(layers.Layer):
    def __init__(self, input_dim=1, num_coefficients=5):
        super(MDNLayer, self).__init__()
        self.num_coefficients = num_coefficients
        self.dense = layers.Dense(64, activation='tanh', name="mdn-dense")
        self.mu = layers.Dense(input_dim * num_coefficients, name="mdn-mu")
        self.logvar = layers.Dense(num_coefficients, name="mdn-logvar")
        self.pi = layers.Dense(num_coefficients, activation='softmax', name="mdn-pi")
        
    def call(self, inputs):
        x = self.dense(inputs)
        mu = self.mu(x)
        logvar = tf.math.exp(self.logvar(x), output_shape=(self.num_coefficients))
        pi = self.pi(x)
        return mu, logvar, pi

In [8]:
class MDNRNN(tf.keras.Model):
    def __init__(self, lstm_units=128):
        super(MDNRNN, self).__init__()
        self.lstm = layers.LSTM(128, name='lstm')
        self.mdnlayer = MDNLayer()
        
    def train_vars(self):
        return self.lstm.trainable_variables + self.mdnlayer.trainable_variables
        
    def call(self, inputs):
        x = self.lstm(x)
        mu, logvar, pi = self.mdnlayer(x)
#         mdn_rnn_loss = self.calculate_mdn_rnn_loss()
#         self.add_loss(mdn_rnn_loss)
        return mu, logvar, pi

In [7]:
def pdf(y, mu, var):
    """Calculate component density"""
    value = tf.subtract(y, mu)**2
    value = (1/tf.math.sqrt(2 * np.pi * var)) * tf.math.exp((-1/(2*var)) * value)
    return value

def calculate_mdn_rnn_loss(y_true, pi, mu, var):
    out = pdf(y_true, mu, var)
    # multiply with each pi and sum it
    out = tf.multiply(out, pi)
    out = tf.reduce_sum(out, 1, keepdims=True)
    out = -tf.math.log(out + 1e-10)
    return tf.reduce_mean(out)


In [9]:
def train_step(model, optimizer, train_x, train_y):
    with tf.GradientTape() as tape:
        mu, logvar, pi = model(train_x)
        loss = calculate_mdn_rnn_loss(train_y, mu, logvar, pi)
    
    gradients = tf.gradients(loss, model.train_vars)
    tf.apply_gradients(zip(gradients, model.train_vars))
    return loss

# Load Data via TF Dataset

In [14]:
# Load npz file
with np.load('') as data:
    npy_data = data['']

FileNotFoundError: [Errno 2] No such file or directory: ''

In [15]:
train_ds = tf.data.Dataset.from_tensor_slices(npy_data)

NameError: name 'npy_data' is not defined

# Train Model

In [10]:
model = MDNRNN()

W0928 16:17:23.648365 140127092012864 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f71633db240>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [11]:
optimizer = tf.keras.optimizers.Adam(1e-4)